In [1]:
import pandas as pd
import numpy as np
import re
import datetime as dt
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from datasets import load_dataset
display(HTML("<style>.container { width:100% !important; }</style>"))
from google.cloud import bigquery
import os
#os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/gikok/code/ingka-search-modelling-dev-3a2a4f44d9c5.json'
import time

In [ ]:
client = bigquery.Client()
query = """
# Retrieves all events with click actions (event_action = select_content),their queries, resulting products, and number of queries resulting in specific product
# Retrieves all events with click actions (event_action = select_content),their queries, and resulting products for the first PLP

SELECT id, name, summary, benefits, key_w FROM(
(
SELECT 
  GLOBAL_ID AS id,
  REGEXP_REPLACE(REGEXP_SUBSTR(REGEXP_SUBSTR(IMAGE_URL, "[^/]+$"),r"(\w.+)+__"),'-',' ') AS name,
  SUMMARY_TEXT_ENGLISH AS summary,
  BENEFIT_LIST AS benefits,
  TYPE_NAME_ENGLISH AS key_w
FROM
  `ingka-rrm-visualinthub-prod.visual_search_artefacts.global_running_range` as global
WHERE
  IMAGE_URL LIKE '%/en/%'
  AND REGEXP_CONTAINS(TYPE_NAME_ENGLISH, r'[a-zA-Z]')
) 
LEFT JOIN
(SELECT GLOBAL_ID, LOCAL_ID FROM `ingka-rrm-visualinthub-prod.visual_search_artefacts.markets_running_range` WHERE COUNTRY_CODE = "gb") as local
ON 
id = local.GLOBAL_ID
  )
WHERE GLOBAL_ID is not null

"""
n = client.query(query).to_dataframe()

In [ ]:
client = bigquery.Client()
query = """
# Retrieves all events with click actions (event_action = select_content),their queries, resulting products, and number of queries resulting in specific product
# Retrieves all events with click actions (event_action = select_content),their queries, and resulting products for the first PLP

WITH session_target AS (
  SELECT
    hit_sequence_number,
    event_category,
    event_action,
    session_id,
    page_url_query_parameter,
    next_page_url,
    SPLIT(REGEXP_EXTRACT(page_url_query_parameter, r'\?q=(\w.+)+'), '&')[SAFE_OFFSET(0)] AS clean_query,
    ARRAY_REVERSE(SPLIT(next_page_url,'/'))[SAFE_OFFSET(0)] AS target_product,
    ARRAY_REVERSE(SPLIT(ARRAY_REVERSE(SPLIT(next_page_url,'/'))[SAFE_OFFSET(0)], '-'))[SAFE_OFFSET(0)] AS target_product_id,
  FROM
    ingka-web-analytics-prod.web_data_v2.hits_events_and_pages
  WHERE
    ( date_hit BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 365 DAY) AND CURRENT_DATE()
      AND website_market_short = 'gb'
      AND event_category = 'engagement'
      AND event_action = 'select_content'
      AND page_url_query_parameter IS NOT NULL) 

),
clean_query_product AS (
  SELECT
    session_id,
    clean_query,
    REGEXP_REPLACE(target_product, r'-', ' ') AS target_product,
    target_product_id,
    COUNT(*) AS query_n_resulting_product
  FROM
    session_target
  WHERE
    clean_query IS NOT NULL
    AND REGEXP_CONTAINS(target_product, r'[0-9A-Za-z]*[\d]')--target_product_id NOT IN ('products', 'en', 'Site Exit', 'gallery', 'shoppingcart', 'latest', 'search', 'login') -- prevent the target_product to become one of these 'next_pages'
  GROUP BY
    clean_query,
    target_product_id,
    target_product,
    session_id
  ORDER BY
    clean_query
)

SELECT 
  clean_query, 
  ARRAY_AGG(target_product_id IGNORE NULLS order by query_n_resulting_product desc) as term_frequency_ranking
FROM clean_query_product 
GROUP BY clean_query

"""
r = client.query(query).to_dataframe()

In [ ]:
df['rank_1'] = df['term_frequency_ranking'].apply(lambda x: re.sub("[^0-9]", "",list(x)[0]).zfill(8))
df['rank_2'] = df['term_frequency_ranking'].apply(lambda x: re.sub("[^0-9]", "",list(x)[1]).zfill(8) if len(list(x))>1 else None)
df['rank_3'] = df['term_frequency_ranking'].apply(lambda x: re.sub("[^0-9]", "",list(x)[2]).zfill(8) if len(list(x))>2 else None)
df['rank_4'] = df['term_frequency_ranking'].apply(lambda x: re.sub("[^0-9]", "",list(x)[3]).zfill(8) if len(list(x))>3 else None)
df['rank_5'] = df['term_frequency_ranking'].apply(lambda x: re.sub("[^0-9]", "",list(x)[4]).zfill(8) if len(list(x))>4 else None)
df['rank_6'] = df['term_frequency_ranking'].apply(lambda x: re.sub("[^0-9]", "",list(x)[5]).zfill(8) if len(list(x))>5 else None)
df['rank_7'] = df['term_frequency_ranking'].apply(lambda x: re.sub("[^0-9]", "",list(x)[6]).zfill(8) if len(list(x))>6 else None)
df['rank_8'] = df['term_frequency_ranking'].apply(lambda x: re.sub("[^0-9]", "",list(x)[7]).zfill(8) if len(list(x))>7 else None)
df['rank_9'] = df['term_frequency_ranking'].apply(lambda x: re.sub("[^0-9]", "",list(x)[8]).zfill(8) if len(list(x))>8 else None)
df['rank_10'] = df['term_frequency_ranking'].apply(lambda x: re.sub("[^0-9]", "",list(x)[9]).zfill(8) if len(list(x))>9 else None)
df.drop(labels='term_frequency_ranking', axis=1, inplace=True)
df['clean_query'] = df['clean_query'].apply(lambda x: x.replace('%20', ' '))
df['clean_query'] = df['clean_query'].apply(lambda x: x.replace('+', ' '))

In [ ]:
r.to_hdf('year_queries.hdf5', index=False, key='year_queries')

In [ ]:
queries

In [2]:
df = pd.read_csv('descriptions.csv')
df

,id,name,summary,benefits,key_w
0,"art,20432499",rensare bag check pattern black,The bags in the RENSARE series are a dream if ...,Perfect for shoes or other larger items you wa...,Bag
1,"art,00473693",knalla bag black white,This bag is just as practical for weekly groce...,"Perfect for transporting most things, from clo...",Bag
2,"art,10514315",kaseberga bag black,This bag takes up little space when folded up ...,The fabric in this product contains recycled p...,Bag
3,"art,60514308",kaseberga bag black white,This black and white bag has room for a lot an...,The bag has 2 removeable compartments – 1 open...,Bag
4,"art,20514305",kaseberga bag multicolour,This colourful bag has room for a lot and is p...,The bag has 2 removeable compartments – 1 open...,Bag
...,...,...,...,...,...
10351,"spr,69459473",metod maximera wall cabinet w glass door 2 dra...,METOD kitchen system gives you endless possibi...,"The drawer closes slowly, quietly and softly t...",Wall cabinet w glass door/2 drawers
10352,"spr,29469817",metod maximera wall cabinet w glass door 2 dra...,METOD kitchen system gives you endless possibi...,"The drawer closes slowly, quietly and softly t...",Wall cabinet w glass door/2 drawers
10353,"spr,39457512",metod maximera wall cabinet w glass door 2 dra...,METOD kitchen system gives you endless possibi...,"The drawer closes slowly, quietly and softly t...",Wall cabinet w glass door/2 drawers
10354,"spr,59455994",metod maximera wall cabinet w glass door 2 dra...,METOD kitchen system gives you endless possibi...,"The drawer closes slowly, quietly and softly t...",Wall cabinet w glass door/2 drawers


In [3]:
df

,id,name,summary,benefits,key_w,item_no
0,"art,20432499",rensare bag check pattern black,The bags in the RENSARE series are a dream if ...,Perfect for shoes or other larger items you wa...,Bag,20432499
1,"art,00473693",knalla bag black white,This bag is just as practical for weekly groce...,"Perfect for transporting most things, from clo...",Bag,00473693
2,"art,10514315",kaseberga bag black,This bag takes up little space when folded up ...,The fabric in this product contains recycled p...,Bag,10514315
3,"art,60514308",kaseberga bag black white,This black and white bag has room for a lot an...,The bag has 2 removeable compartments – 1 open...,Bag,60514308
4,"art,20514305",kaseberga bag multicolour,This colourful bag has room for a lot and is p...,The bag has 2 removeable compartments – 1 open...,Bag,20514305
...,...,...,...,...,...,...
10351,"spr,69459473",metod maximera wall cabinet w glass door 2 dra...,METOD kitchen system gives you endless possibi...,"The drawer closes slowly, quietly and softly t...",Wall cabinet w glass door/2 drawers,69459473
10352,"spr,29469817",metod maximera wall cabinet w glass door 2 dra...,METOD kitchen system gives you endless possibi...,"The drawer closes slowly, quietly and softly t...",Wall cabinet w glass door/2 drawers,29469817
10353,"spr,39457512",metod maximera wall cabinet w glass door 2 dra...,METOD kitchen system gives you endless possibi...,"The drawer closes slowly, quietly and softly t...",Wall cabinet w glass door/2 drawers,39457512
10354,"spr,59455994",metod maximera wall cabinet w glass door 2 dra...,METOD kitchen system gives you endless possibi...,"The drawer closes slowly, quietly and softly t...",Wall cabinet w glass door/2 drawers,59455994


In [4]:
new_df = df[['item_no']+list(df.columns[1:-1])]
new_df

,item_no,name,summary,benefits,key_w
0,20432499,rensare bag check pattern black,The bags in the RENSARE series are a dream if ...,Perfect for shoes or other larger items you wa...,Bag
1,00473693,knalla bag black white,This bag is just as practical for weekly groce...,"Perfect for transporting most things, from clo...",Bag
2,10514315,kaseberga bag black,This bag takes up little space when folded up ...,The fabric in this product contains recycled p...,Bag
3,60514308,kaseberga bag black white,This black and white bag has room for a lot an...,The bag has 2 removeable compartments – 1 open...,Bag
4,20514305,kaseberga bag multicolour,This colourful bag has room for a lot and is p...,The bag has 2 removeable compartments – 1 open...,Bag
...,...,...,...,...,...
10351,69459473,metod maximera wall cabinet w glass door 2 dra...,METOD kitchen system gives you endless possibi...,"The drawer closes slowly, quietly and softly t...",Wall cabinet w glass door/2 drawers
10352,29469817,metod maximera wall cabinet w glass door 2 dra...,METOD kitchen system gives you endless possibi...,"The drawer closes slowly, quietly and softly t...",Wall cabinet w glass door/2 drawers
10353,39457512,metod maximera wall cabinet w glass door 2 dra...,METOD kitchen system gives you endless possibi...,"The drawer closes slowly, quietly and softly t...",Wall cabinet w glass door/2 drawers
10354,59455994,metod maximera wall cabinet w glass door 2 dra...,METOD kitchen system gives you endless possibi...,"The drawer closes slowly, quietly and softly t...",Wall cabinet w glass door/2 drawers


In [ ]:
new_df.to_hdf('all_en_descriptions.hdf5', key='all')

In [ ]:
b.to_hdf('filtered_GB_en_descriptions.hdf5', key='all')

In [ ]:
b = new_df.merge(f, how='inner', on='item_no')

In [ ]:
df.nunique()

In [ ]:
t = pd.DataFrame({'item_no':[], 'input':[], 'target':[], 'options':[]})

In [ ]:
t = t.append({'item_no':item_no, 'input':inp, 'target':target, 'options':opt}, ignore_index=True)
t

In [ ]:
functions = [no_to_name, name_to_no, is_description, is_summary, true_query, true_query, true_query, query_rank, query_rank]


In [ ]:
ls -t

In [ ]:
np.random.seed(12)
#queries = pd.read_hdf('year_queries.hdf5')
#df = pd.read_hdf("filtered_GB_en_descriptions.hdf5")
functions = [no_to_name, name_to_no, is_description, is_summary, true_query, true_query, true_query, query_rank, query_rank]
prompt_df = pd.DataFrame({'global_index':[],'item_no':[], 'input':[], 'target':[], 'options':[]})
for i in range(len(df)):
    start = time.time()
    print('iteration: ', i)
    index = i
    item_no = df['item_no'].iloc[index]
    query_df = queries[queries.isin([item_no]).any(axis=1)]
    r = np.random.randint(0, 9, size=1024)
    inputs = []
    targets = []
    options = []
    for j in range(1024):
        f = functions[r[j]]
        
        if r[j] <4:
            inp, tar, opt = f(index)
            inputs += [inp]
            targets += [tar]
            options += [opt]
        
        else:
            inp, tar, opt = f(query_df, item_no)
            inputs += [inp]
            targets += [tar]
            options += [opt]
        
    prompt_df = prompt_df.append(pd.DataFrame({'global_index':list(np.arange(1024*i, 1024*(i+1))),'item_no':1024*[item_no], 'input':inputs, 'target':targets, 'options':options}), ignore_index=True)
    print(f"generating prompts for item_no {item_no} took {np.round(time.time()-start)} seconds.")
    print("")
    
prompt_df.to_hdf('prompts.hdf5', key='alpha')
prompt_df['insertion_time'] = dt.datetime.now()
prompt_df['options'] = prompt_df['options'].apply(lambda x: str(x))
prompt_df['global_index'] = prompt_df['global_index'].astype(int)
prompt_df.to_gbq(destination_table = 'prompts.prompts_001', project_id='ingka-search-modelling-dev', if_exists='append')

In [ ]:
df = pd.read_hdf('prompts.hdf5')

In [ ]:
df

In [ ]:
df.to_parquet('df.parquet.gzip',
              compression='gzip')

In [ ]:
ls

In [ ]:
df = pd.read_hdf("../data/filtered_GB_en_descriptions.hdf5")
d = df['item_no'].drop_duplicates().reset_index(drop=True)
pd.DataFrame(d).to_parquet('../data/item_no_6k.gzip')

In [ ]:
data = pd.read_parquet('../training/data/prompts_001.gzip')

In [ ]:
data['len'] = data['input'].apply(lambda x: len(x))

In [ ]:
plt.hist(data['len'], 40)
plt.show()

In [ ]:
def tokenize_train(examples):
    input_texts = examples['input']
    target_texts = examples['target']
    opts = examples['options']
    print(input_texts[0])
    print(type(input_texts), len(input_texts))
    print("lol")
    return input_texts
train_dataset = data.map(
    tokenize_train, batched=True
)

In [ ]:
start = time.time()
inp, target, opt = no_to_name(index)
item_no = df['item_no'].iloc[0]
print("################# PROMPT TYPE 1 ########################")
print("INPUT:",inp)
print("")
print("")
print("TARGET:", target)
inp, target, opt = name_to_no(index)
print("################# PROMPT TYPE 2 ########################")
print("INPUT:",inp)
print("")
print("")
print("TARGET:", target)
inp, target, opt = is_description(index)
print("################# PROMPT TYPE 3 ########################")
print("INPUT:",inp)
print("")
print("")
print("TARGET:", target)
inp, target, opt = is_summary(index)
print("################# PROMPT TYPE 4 ########################")
print("INPUT:",inp)
print("")
print("")
print("TARGET:", target)   
# get queries that have item_no
query_df = queries[queries.isin([item_no]).any(axis=1)]
inp, target, opt = true_query(query_df, item_no)
print("################# PROMPT TYPE 5 ########################")
print("INPUT:",inp)
print("")
print("")
print("TARGET:", target)
inp, target, opt = query_rank(query_df, item_no)
print("################# PROMPT TYPE 6 ########################")
print("INPUT:",inp)
print("")
print("")
print("TARGET:", target)
end = time.time()
print(f"It took {(end-start)*1e3} ms to generate prompts")


In [ ]:
type(np.arange(3)[0])

In [ ]:
n[n['item_no']=='00248567']

In [ ]:
def add_or(s):
    li = s.rsplit(', ')
    start = ', '.join(li[:-1])
    end =  ' or ' + li[-1]
    return start+end
    

def create_option_string(df, column, inds):
    options = ""
    for i in inds:
        options +=  f"'{df[column].iloc[i]}', "
        if i == inds[-1]:
            options = options[:-2]
            
    option_string = add_or(options)
    option_list = options.split(',')
        
    return option_string, option_list

def get_other_indices(index, size, num):
    """
    given a number {index}, returns {num} integers in range({size})
    """
    
    proceed = 0
    while proceed == 0:
        inds = np.random.choice(range(size), num)
        # check if any of the new indices is same as initial
        if (inds == index).any()==False:
            proceed = 1
        else:
            proceed = 0
    
    return inds


def no_to_name(index):
    
    # get random number of options
    num = np.random.randint(1, 4)
    
    # get random item index
    inds = get_other_indices(index, len(df), num)
    inds = np.append(inds, index)
    np.random.shuffle(inds)
    
    # get answer options
    options, option_list = create_option_string(df, 'name', inds)
    
    # create the input string
    inp = f"if item_no is {df['item_no'].iloc[index]}, which of the following is the correct name: " + options + '?'
    
    
    # set the target
    target = df['name'].iloc[inds[np.argmin(inds-index)]]
    
    return inp.lower(), target.lower(), option_list

def name_to_no(index):
    
    # get random number of options
    num = np.random.randint(1, 4)
    
    # get random item index
    inds = get_other_indices(index, len(df), num)
    inds = np.append(inds, index)
    np.random.shuffle(inds)
    
    # get answer options
    options, option_list = create_option_string(df, 'item_no', inds)
    
    # get the input string
    inp = f"if name is {df['name'].iloc[index]}, what item_no does it refer to? " + options + '?' 
    
    # get the target
    target = df['item_no'].iloc[inds[np.argmin(inds-index)]]
    
    return inp.lower(), target.lower(), option_list

def is_description(index):
    
    is_true = np.random.randint(2)
    
    if is_true:
        desc = df['benefits'].iloc[index]
        target = "yes"
    else:
        tempdf = df[df['benefits']!=df['benefits'].iloc[index]]
        desc = np.random.choice(tempdf['benefits'])
        target = "no"
        
    # make sure ends with period    
    desc = desc if desc.endswith(".") else desc + "."
    
    # create input
    inp = desc + f" is the previous sentence a description of item_no {df['item_no'].iloc[index]}. yes or no?"
    
    return inp.lower(), target, ['yes', 'no']

def is_summary(index):
    
    is_true = np.random.randint(2)
    
    if is_true:
        desc = df['key_w'].iloc[index]
        target = "yes"
    else:
        tempdf = df[df['key_w']!=df['key_w'].iloc[index]]
        desc = np.random.choice(tempdf['key_w'])
        target = "no"
        
    # make sure ends with period    
    desc = desc if desc.endswith(".") else desc + "."
    
    # create input
    inp = desc + f" is the previous sentence a summary of item_no {df['item_no'].iloc[index]}. yes or no?"
    
    return inp.lower(), target, ['yes', 'no']

def true_query(query_df, item_no):
    
    is_true = np.random.randint(2)
    
    if is_true:
        query = query_df['clean_query'].sample().iloc[0]
        target = "yes"
    else:
        query = query_df['clean_query'].sample().iloc[0]
        target = "no"

    
    # create input
    inp = "query:'" + query + f"'\ndoes the query above return item_no {item_no} as a result. yes or no?"
    
    return inp.lower(), target, ['yes', 'no']

def query_rank(query_df, item_no):

    # select random row
    row = query_df.sample()
    
    # get query string
    query = row['clean_query'].iloc[0]
    
    # get rank of item_no
    cols = np.array(row.columns)
    inds = (row[row==item_no].isnull()==False).values[0]
    
    # get first true value, that is not 0 (queries can be for item_no) and
    # sometimes same item_no has multiple ranks
    if len(inds)>1:
        inds[0] = False
        rank = cols[np.argmax(inds)][-1]
        
        # if doesn't work keep trying
        c = 0
        while rank.isdigit()==False:
            row = query_df.sample()
            inds = (row[row==item_no].isnull()==False).values[0]
            inds[0] = False
            rank = cols[np.argmax(inds)][-1]
            c +=1
            if c>100:
                print(f'stuck in infinite loop with item_no {item_no}')
                break
            
    
    # in case rank is 10
    rank = '10' if rank=='0' else rank
    
    # create possible options
    n_options = np.random.randint(1, 5)
    
    options = list(np.arange(1, 11))
    options.remove(int(rank))
    
    options = np.append(np.random.choice(options, n_options).astype(str), rank)
    np.random.shuffle(options)
    
    # create input
    inp = "query:'" + query + f"'\nthe query above returns item_no {item_no} as a result. what is its rank? " + add_or(', '.join(options))
    
    target = rank
    return inp.lower(), target, list(options)

In [ ]:
l = '1'
l.isdigit()

In [ ]:
df.drop(labels = 'term_frequency_ranking', axis = 1, inplace=True)

In [ ]:
n['item_no'] = n['target_product_id'].copy()

In [ ]:
n['item_no'] = n['target_product_id'].apply(lambda x: re.sub("[^0-9]", "",str(x)).zfill(8))

In [ ]:
f = pd.DataFrame()

In [ ]:
f['item_no'] = pd.unique(queries[queries.columns[1:]].values.ravel('K'))

In [ ]:
prompts

In [ ]:
t.apply({'test':'test', 'testtest':"testtest"})

In [ ]:
raw_train_dataset['train']

In [ ]:
raw_train_dataset = load_dataset('super_glue', 'cb', split='train')
prompts = DatasetTemplates('super_glue', "cb")
template = prompts['consider always/sometimes/never']

column_names = raw_train_dataset.column_names
def preprocess_train(examples):
    bs = len(examples[column_names[0]])

    input_texts = []
    target_texts = []
    answer_choices_texts = []
    for i in range(10):
        ex = {
            k: examples[k][i]
            for k in column_names
        }
        input, target = template.apply(ex)
        ex_answer_choices = template.get_answer_choices_list(ex)
        assert target in ex_answer_choices
        input_texts.append(input)
        target_texts.append(target)
        answer_choices_texts.append(ex_answer_choices)
    return input_texts, target_texts, answer_choices_texts

i, t, a = preprocess_train(raw_train_dataset)
for j in range(10):
    print("%r"%i[j])
    print("%r"%t[j])
    print("%r"%a[j])
    print("********************")

In [ ]:
# get queries that have item_no
qs = queries[queries.isin([item_no]).any(axis=1)]

# select random row
row = qs.sample()

# get query string
query = queries['clean_query']

# get rank of item_no
cols = np.array(row.columns)
inds = (row[row==item_no].isnull()==False).values[0]

In [ ]:
item_no

In [ ]:
row

In [ ]:
inds

In [ ]:
item_no